## Usage

This file is used to plot Supplementary figures (2, 15, 16, 17). For figures 15, 16, 17, we run the run_test_case.py for varying crossover and mutation probabilities (24 combinations in total) repeatedly (10 seeds). Without parallelization, it takes roughly a day of computation. With parallelization, 

In [1]:
from pymoo.indicators.hv import HV
import sys
import json
import numpy as np
import pickle
from multiprocessing import Pool
from amplifier_problem import Amplifier
from signal_conditioner_problem import SignalConditioner
from pulse_generator_problem import PulseGenerator
from load_files_pop import Z_200
from load_Z_mat_samples import Z_mat_list, Ref_list
from saving import make_main_directory
from GA import sampling
from GA_setup import (
    single_obj_GA,
    multi_obj_GA
)
import os
import matplotlib.pyplot as plt

In [2]:
from plot_search_results import(
    plot_graph,
    plot_metric,
    plot_1D_obj_scatter,
    plot_pareto_front,
    plot_pareto_front3D,
    plot_hypervolume
)

In [3]:
def get_foldertag(test_case="Amplifier", DsRed=True, pop_size=200, pop_ratio=0.25, 
                  n_gen=120, p_cross=1., p_mut=1., p_scheme=[0.25, 0.25, 0.25, 0.25]):

    folder_tag = "%s_single" % test_case
    if DsRed:
        folder_tag += "_DsRED"
    folder_tag += "_pop%d_ratio%s" % (pop_size, pop_ratio)
    folder_tag += "_ngen%d" % n_gen
    folder_tag += "_cross%s" % p_cross
    folder_tag += "_mut%s" % p_mut
    folder_tag += "_mscheme_%s" % "_".join([str(_) for _ in p_scheme])

    return folder_tag

In [4]:
with open("./combo_search/amplifier_ranked/ranked_obj_dsred1_final.pkl", "rb") as fid:
    ranked_obj = pickle.load(fid)

with open("./combo_search/amplifier_ranked/ranked_topo_dsred1_final.pkl", "rb") as fid:
    ranked_topo = pickle.load(fid)

In [5]:
all_unique_topos = []
with open("./combo_search/main_combo_1.pkl", "rb") as fid:
    all_unique_topos.extend(pickle.load(fid))

with open("./combo_search/main_combo_2.pkl", "rb") as fid:
    all_unique_topos.extend(pickle.load(fid))

with open("./combo_search/main_combo_2_inhibitor.pkl", "rb") as fid:
    all_unique_topos.extend(pickle.load(fid))

In [6]:
all_unique_topos_dict = dict()
dupes = dict()
for i, topo in enumerate(all_unique_topos):
    part_list = tuple(sorted(topo.part_list))
    if part_list not in all_unique_topos_dict:
        all_unique_topos_dict[part_list] = dict()
    edge_list = tuple(sorted(topo.edge_list))
    if edge_list not in all_unique_topos_dict[part_list]:
        all_unique_topos_dict[part_list][edge_list] = i
    else:
        if all_unique_topos_dict[part_list][edge_list] not in dupes:
            dupes[all_unique_topos_dict[part_list][edge_list]] = []
        dupes[all_unique_topos_dict[part_list][edge_list]].append(i)

In [7]:
total_n_unique_topos = 0
for part_list in all_unique_topos_dict:
    total_n_unique_topos += len(all_unique_topos_dict[part_list])

In [8]:
solution_topo = all_unique_topos_dict[tuple(sorted(ranked_topo.part_list))][tuple(sorted(ranked_topo.edge_list))]

In [9]:
all_unique_topos_dict_dose = dict()
dose_combo_2 = [tuple([i, j]) for i in range(5, 76, 5) for j in range(5, 76, 5)]
i = 0
for topo in all_unique_topos:
    part_list = tuple(sorted(topo.part_list))
    if part_list not in all_unique_topos_dict_dose:
        all_unique_topos_dict_dose[part_list] = dict()
    edge_list = tuple(sorted(topo.edge_list))
    if edge_list not in all_unique_topos_dict_dose[part_list]:
        if i < 360:
            all_unique_topos_dict_dose[part_list][edge_list] = {tuple([dose]): i+dose_ind for dose_ind, dose in enumerate(range(5, 76, 5))}
            i += 15
        else:
            all_unique_topos_dict_dose[part_list][edge_list] = {dose: i+dose_ind for dose_ind, dose in enumerate(dose_combo_2)}
            i += len(dose_combo_2)

In [10]:
total_n_unique_topos_dose = 0
for part_list in all_unique_topos_dict_dose:
    for edge_list in all_unique_topos_dict_dose[part_list]:
        total_n_unique_topos_dose += len(all_unique_topos_dict_dose[part_list][edge_list])

In [11]:
solution_topo_dose = all_unique_topos_dict_dose[tuple(sorted(ranked_topo.part_list))][tuple(sorted(c.edge_list))][tuple([ranked_topo.dose[part] for part in sorted(ranked_topo.part_list)])]

In [16]:
print(f"# unique topologies in the combinatorial search space: {total_n_unique_topos_dose}")

# unique topologies in the combinatorial search space: 1679760


In [17]:
def analyze_geno(folder_name, seed):
    ga_unique_topos = {"parents": [], "children": []}
    ga_unique_topos_dose = {"parents": [], "children": []}
    first_appear = None
    first_appear_dose = None
                       
    all_topos = {"parents": None, "children": None}
    if "Amplifier" in folder_name:
        with open("/scratch/avn2126/GraphGA/GA_results/%s_seed%d/parent_circuits.pkl" % (folder_name, seed), "rb") as fid:
            all_topos["parents"] = pickle.load(fid)
        with open("/scratch/avn2126/GraphGA/GA_results/%s_seed%d/children_circuits.pkl" % (folder_name, seed), "rb") as fid:
            all_topos["children"] = pickle.load(fid)
    elif "SignalConditioner" in folder_name:
        with open("/scratch/avn2126/GraphGA/GA_results/%s_seed%d/top_circuits.pkl" % (folder_name, seed), "rb") as fid:
            all_topos["parents"] = pickle.load(fid)
        with open("/scratch/avn2126/GraphGA/GA_results/%s_seed%d/all_circuits.pkl" % (folder_name, seed), "rb") as fid:
            all_topos["children"] = pickle.load(fid)

    for it in range(120):
        tmp_parents = []
        tmp_parents_dose = []
        tmp_children = []
        tmp_children_dose = []
        for topo in all_topos["parents"][it]:
            part_list = tuple(sorted(topo[0].part_list))
            edge_list = tuple(sorted(topo[0].edge_list))
            dose_list = [topo[0].dose[k] for k in part_list]
            tmp_parents.append(all_unique_topos_dict[part_list][edge_list])
            tmp_parents_dose.append(all_unique_topos_dict_dose[part_list][edge_list][tuple(dose_list)])
            if (first_appear is None) and (all_unique_topos_dict[part_list][edge_list] == solution_topo):
                first_appear = it
            if (first_appear_dose is None) and (all_unique_topos_dict_dose[part_list][edge_list][tuple(dose_list)] == solution_topo_dose):
                first_appear_dose = it
        ga_unique_topos["parents"].append(tmp_parents)
        ga_unique_topos_dose["parents"].append(tmp_parents_dose)
        for topo in all_topos["children"][it]:
            part_list = tuple(sorted(topo[0].part_list))
            edge_list = tuple(sorted(topo[0].edge_list))
            dose_list = tuple([topo[0].dose[k] for k in part_list])
            tmp_children.append(all_unique_topos_dict[part_list][edge_list])
            tmp_children_dose.append(all_unique_topos_dict_dose[part_list][edge_list][dose_list])
        ga_unique_topos["children"].append(tmp_children)
        ga_unique_topos_dose["children"].append(tmp_children_dose)
    
    it = 120
    tmp_parents = []
    tmp_parents_dose = []
    for topo in all_topos["parents"][it]:
        part_list = tuple(sorted(topo[0].part_list))
        edge_list = tuple(sorted(topo[0].edge_list))
        dose_list = tuple([topo[0].dose[k] for k in part_list])
        tmp_parents.append(all_unique_topos_dict[part_list][edge_list])
        tmp_parents_dose.append(all_unique_topos_dict_dose[part_list][edge_list][tuple(dose_list)])
    ga_unique_topos["parents"].append(tmp_parents)
    ga_unique_topos_dose["parents"].append(tmp_parents_dose)
    
    for k, v in ga_unique_topos.items():
        ga_unique_topos[k] = np.asarray(v)
    for k, v in ga_unique_topos_dose.items():
        ga_unique_topos_dose[k] = np.asarray(v)

    return ga_unique_topos, ga_unique_topos_dose, first_appear, first_appear_dose

In [ ]:
folder_name_amp = get_foldertag(test_case="Amplifier", p_cross=0.55, p_mut=1., pop_size=52, pop_ratio=0.5)
folder_name_sigcond = get_foldertag(test_case="SignalConditioner", p_cross=0.06, p_mut=0.99, pop_size=200, pop_ratio=0.25)
for folder_name in [folder_name_amp, folder_name_sigcond]:
    print(folder_name)
    for seed in range(10):
        ga_unique_topos, ga_unique_topos_dose, first_appear, first_appear_dose = analyze_geno(folder_name, seed)
        n_unique_topos = {"parents": [], "children": [], "all": [], "overall": []}
        for it in range(120):
            n_unique_topos["parents"].append(len(np.unique(ga_unique_topos["parents"][it])))
            n_unique_topos["children"].append(len(np.unique(ga_unique_topos["children"][it])))
            n_unique_topos["all"].append(len(np.unique(np.append(ga_unique_topos["parents"][it], ga_unique_topos["children"][it]))))
        it = 120
        n_unique_topos["parents"].append(len(np.unique(ga_unique_topos["parents"][it])))
        track = []
        for it in range(120):
            tmp = set(list(np.append(ga_unique_topos["parents"][it], ga_unique_topos["children"][it])))
            tmp2 = tmp.difference(set(track))
            n_unique_topos["overall"].append(len(tmp2))
            track.extend(list(tmp2))
        for k, v in n_unique_topos.items():
            n_unique_topos[k] = np.asarray(v)

        n_unique_topos_dose = {"parents": [], "children": [], "all": [], "overall": []}
        for it in range(120):
            n_unique_topos_dose["parents"].append(len(np.unique(ga_unique_topos_dose["parents"][it])))
            n_unique_topos_dose["children"].append(len(np.unique(ga_unique_topos_dose["children"][it])))
            n_unique_topos_dose["all"].append(len(np.unique(np.append(ga_unique_topos_dose["parents"][it], ga_unique_topos_dose["children"][it]))))
        it = 120
        n_unique_topos_dose["parents"].append(len(np.unique(ga_unique_topos_dose["parents"][it])))
        
        track = []
        for it in range(120):
            tmp = set(list(np.append(ga_unique_topos_dose["parents"][it], ga_unique_topos_dose["children"][it])))
            tmp2 = tmp.difference(set(track))
            n_unique_topos_dose["overall"].append(len(tmp2))
            track.extend(list(tmp2))
        for k, v in n_unique_topos_dose.items():
            n_unique_topos_dose[k] = np.asarray(v)

        with open("Analysis/%s_seed%d.pkl" % (folder_name, seed), "wb") as fid:
            pickle.dump([n_unique_topos, n_unique_topos_dose, first_appear, first_appear_dose], fid)

In [ ]:
geno_cumulative = {"amplifier": np.zeros(10), "sigcond": np.zeros(10)}
geno_cumulative_dose = {"amplifier": np.zeros(10), "sigcond": np.zeros(10)}

folder_name = get_foldertag(test_case="Amplifier", p_cross=0.55, p_mut=1., pop_size=52, pop_ratio=0.5)

for seed in range(10):
    with open("Analysis/%s_seed%d.pkl" % (folder_name, seed), "rb") as fid:
        [n_unique_topos, n_unique_topos_dose, first_appear, first_appear_dose] = pickle.load(fid)
    geno_cumulative["amplifier"][seed] = np.sum(n_unique_topos["overall"])
    geno_cumulative_dose["amplifier"][seed] = np.sum(n_unique_topos_dose["overall"])

folder_name = get_foldertag(test_case="SignalConditioner", p_cross=0.06, p_mut=0.99, pop_size=200, pop_ratio=0.25)
for seed in range(10):
    with open("Analysis/%s_seed%d.pkl" % (folder_name, seed), "rb") as fid:
        [n_unique_topos, n_unique_topos_dose, first_appear, first_appear_dose] = pickle.load(fid)
    geno_cumulative["sigcond"][seed] = np.sum(n_unique_topos["overall"])
    geno_cumulative_dose["sigcond"][seed] = np.sum(n_unique_topos_dose["overall"])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4.5))
ax.boxplot(geno_cumulative_dose["amplifier"]/total_n_unique_topos_dose*100, positions=[0]);
ax.boxplot(geno_cumulative_dose["sigcond"]/total_n_unique_topos_dose*100, positions=[1]);
ax.set_xticks([0, 1], ["Amplifier", "Signal Conditioner"])
ax.set_ylabel("Percentage", fontsize=20)
ax.tick_params(axis='x', labelsize=20)
ax.tick_params(axis='y', labelsize=15)
# ax.set_xlabel("(b) With dose", fontsize=20)
ax.grid()
# plt.tight_layout()

In [ ]:
geno_parents = dict()
geno_children = dict()
geno_combine = dict()
geno_cumulative = dict()
geno_parents_dose = dict()
geno_children_dose = dict()
geno_combine_dose = dict()
geno_cumulative_dose = dict()
geno_first = dict()
geno_first_dose = dict()
geno_conv = dict()
geno_conv_dose = dict()
for p_cross in [0., 0.1, 0.3, 0.5, 0.7, 0.9, 1.]:
    for p_mut in [0., 0.1, 0.3, 0.5, 0.7, 0.9, 1.]:
        if (p_cross == 0.) and (p_mut == 0.):
            continue
        folder_name = get_foldertag(p_cross=p_cross, p_mut=p_mut)
        geno_parents[(p_cross, p_mut)] = []
        geno_parents_dose[(p_cross, p_mut)] = []
        geno_children[(p_cross, p_mut)] = []
        geno_children_dose[(p_cross, p_mut)] = []
        geno_combine[(p_cross, p_mut)] = []
        geno_combine_dose[(p_cross, p_mut)] = []
        geno_cumulative[(p_cross, p_mut)] = []
        geno_cumulative_dose[(p_cross, p_mut)] = []
        geno_first[(p_cross, p_mut)] = []
        geno_first_dose[(p_cross, p_mut)] = []
        geno_conv[(p_cross, p_mut)] = []
        geno_conv_dose[(p_cross, p_mut)] = []
        for seed in range(10):
            with open("Analysis/%s_seed%d.pkl" % (folder_name, seed), "rb") as fid:
                [n_unique_topos, n_unique_topos_dose, first_appear, first_appear_dose] = pickle.load(fid)
            geno_first[(p_cross, p_mut)].append(first_appear if first_appear is not None else 0)
            geno_first_dose[(p_cross, p_mut)].append(first_appear_dose if first_appear_dose is not None else 0)
            geno_parents[(p_cross, p_mut)].append(np.mean(n_unique_topos["parents"]))
            geno_parents_dose[(p_cross, p_mut)].append(np.mean(n_unique_topos_dose["parents"]))
            geno_children[(p_cross, p_mut)].append(np.mean(n_unique_topos["children"]))
            geno_children_dose[(p_cross, p_mut)].append(np.mean(n_unique_topos_dose["children"]))
            geno_combine[(p_cross, p_mut)].append(np.mean(n_unique_topos["all"]))
            geno_combine_dose[(p_cross, p_mut)].append(np.mean(n_unique_topos_dose["all"]))
            geno_cumulative[(p_cross, p_mut)].append(np.sum(n_unique_topos["overall"]))
            geno_cumulative_dose[(p_cross, p_mut)].append(np.sum(n_unique_topos_dose["overall"]))
            geno_conv[(p_cross, p_mut)].append(np.argwhere(n_unique_topos["parents"] == 1)[0][0])
            geno_conv_dose[(p_cross, p_mut)].append(np.argwhere(n_unique_topos_dose["parents"] == 1)[0][0])

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 12), layout="tight", sharey=True)
for i, p in enumerate([0.1, 0.3, 0.5, 0.7, 0.9, 1.]):
    bp = axes[0, 0].boxplot(np.asarray(geno_parents[(p, 0.)])/200*100, positions=[i], widths=[0.2], 
                    boxprops=dict(linewidth=2, alpha=0.7),
                    whiskerprops=dict(linewidth=2, alpha=0.7),
                    capprops=dict(linewidth=2, alpha=0.7),
                    medianprops=dict(linewidth=0, color='r'), 
                    flierprops=dict(markersize=8, linewidth=2), 
                    showmeans=True, meanprops=dict(markersize=10, color='m'));
    bp = axes[0, 1].boxplot(np.asarray(geno_parents[(0., p)])/200*100, positions=[i], widths=[0.2], 
                    boxprops=dict(linewidth=2, alpha=0.7),
                    whiskerprops=dict(linewidth=2, alpha=0.7),
                    capprops=dict(linewidth=2, alpha=0.7),
                    medianprops=dict(linewidth=0, color='r'), 
                    flierprops=dict(markersize=8, linewidth=2), 
                    showmeans=True, meanprops=dict(markersize=10, color='m'));
for i, p in enumerate([0.1, 0.3, 0.5, 0.7, 0.9, 1.]):
    bp = axes[1, 0].boxplot(np.asarray(geno_parents[(p, 1.)])/200*100, positions=[i], widths=[0.2], 
                    boxprops=dict(linewidth=2, alpha=0.7),
                    whiskerprops=dict(linewidth=2, alpha=0.7),
                    capprops=dict(linewidth=2, alpha=0.7),
                    medianprops=dict(linewidth=0, color='r'), 
                    flierprops=dict(markersize=8, linewidth=2), 
                    showmeans=True, meanprops=dict(markersize=10, color='m'));
    bp = axes[1, 1].boxplot(np.asarray(geno_parents[(1., p)])/200*100, positions=[i], widths=[0.2], 
                    boxprops=dict(linewidth=2, alpha=0.7),
                    whiskerprops=dict(linewidth=2, alpha=0.7),
                    capprops=dict(linewidth=2, alpha=0.7),
                    medianprops=dict(linewidth=0, color='r'), 
                    flierprops=dict(markersize=8, linewidth=2), 
                    showmeans=True, meanprops=dict(markersize=10, color='m'));

axes[0, 0].set_title("p_c varies, p_m=0", fontsize=20)
axes[0, 1].set_title("p_m varies, p_c=0", fontsize=20)
axes[1, 0].set_title("p_c varies, p_m=1", fontsize=20)
axes[1, 1].set_title("p_m varies, p_c=1", fontsize=20)

for i in range(2):
    for j in range(2):
        axes[i, j].grid()
        axes[i, j].set_xticks([0, 1, 2, 3, 4, 5], [0.1, 0.3, 0.5, 0.7, 0.9, 1.])
        axes[i, j].set_xlabel("Probability", fontsize=20)
        axes[i, j].tick_params(axis='both', labelsize=15)
        # axes[i].legend([bp['medians'][0], bp['means'][0]], ['median', 'mean'], fontsize=20);
        axes[i, j].legend([bp['means'][0]], ['mean'], fontsize=20);
axes[0, 0].set_ylabel("Average pct.", fontsize=20);
axes[1, 0].set_ylabel("Average pct.", fontsize=20);

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 12), layout="tight", sharey=True)
for i, p in enumerate([0.1, 0.3, 0.5, 0.7, 0.9, 1.]):
    bp = axes[0, 0].boxplot(np.asarray(geno_conv_dose[(p, 0.)]), positions=[i], widths=[0.2], 
                    boxprops=dict(linewidth=2, alpha=0.7),
                    whiskerprops=dict(linewidth=2, alpha=0.7),
                    capprops=dict(linewidth=2, alpha=0.7),
                    medianprops=dict(linewidth=0, color='r'), 
                    flierprops=dict(markersize=8, linewidth=2), 
                    showmeans=True, meanprops=dict(markersize=10, color='m'));
    bp = axes[0, 1].boxplot(np.asarray(geno_conv_dose[(0., p)]), positions=[i], widths=[0.2], 
                    boxprops=dict(linewidth=2, alpha=0.7),
                    whiskerprops=dict(linewidth=2, alpha=0.7),
                    capprops=dict(linewidth=2, alpha=0.7),
                    medianprops=dict(linewidth=0, color='r'), 
                    flierprops=dict(markersize=8, linewidth=2), 
                    showmeans=True, meanprops=dict(markersize=10, color='m'));
for i, p in enumerate([0.1, 0.3, 0.5, 0.7, 0.9, 1.]):
    bp = axes[1, 0].boxplot(np.asarray(geno_conv_dose[(p, 1.)]), positions=[i], widths=[0.2], 
                    boxprops=dict(linewidth=2, alpha=0.7),
                    whiskerprops=dict(linewidth=2, alpha=0.7),
                    capprops=dict(linewidth=2, alpha=0.7),
                    medianprops=dict(linewidth=0, color='r'), 
                    flierprops=dict(markersize=8, linewidth=2), 
                    showmeans=True, meanprops=dict(markersize=10, color='m'));
    bp = axes[1, 1].boxplot(np.asarray(geno_conv_dose[(1., p)]), positions=[i], widths=[0.2], 
                    boxprops=dict(linewidth=2, alpha=0.7),
                    whiskerprops=dict(linewidth=2, alpha=0.7),
                    capprops=dict(linewidth=2, alpha=0.7),
                    medianprops=dict(linewidth=0, color='r'), 
                    flierprops=dict(markersize=8, linewidth=2), 
                    showmeans=True, meanprops=dict(markersize=10, color='m'));

axes[0, 0].set_title("p_c varies, p_m=0", fontsize=20)
axes[0, 1].set_title("p_m varies, p_c=0", fontsize=20)
axes[1, 0].set_title("p_c varies, p_m=1", fontsize=20)
axes[1, 1].set_title("p_m varies, p_c=1", fontsize=20)

for i in range(2):
    for j in range(2):
        axes[i, j].grid()
        axes[i, j].set_xticks([0, 1, 2, 3, 4, 5], [0.1, 0.3, 0.5, 0.7, 0.9, 1.])
        axes[i, j].set_xlabel("Probability", fontsize=20)
        axes[i, j].tick_params(axis='both', labelsize=15)
        # axes[i].legend([bp['medians'][0], bp['means'][0]], ['median', 'mean'], fontsize=20);
        axes[i, j].legend([bp['means'][0]], ['mean'], fontsize=20);
axes[0, 0].set_ylabel("Generation", fontsize=20);
axes[1, 0].set_ylabel("Generation", fontsize=20);

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(20, 12), layout="tight", sharey=True)
for i, p in enumerate([0.1, 0.3, 0.5, 0.7, 0.9, 1.]):
    bp = axes[0, 0].boxplot(np.asarray(geno_first_dose[(p, 0.)]), positions=[i], widths=[0.2], 
                    boxprops=dict(linewidth=2, alpha=0.7),
                    whiskerprops=dict(linewidth=2, alpha=0.7),
                    capprops=dict(linewidth=2, alpha=0.7),
                    medianprops=dict(linewidth=0, color='r'), 
                    flierprops=dict(markersize=8, linewidth=2), 
                    showmeans=True, meanprops=dict(markersize=10, color='m'));
    bp = axes[0, 1].boxplot(np.asarray(geno_first_dose[(0., p)]), positions=[i], widths=[0.2], 
                    boxprops=dict(linewidth=2, alpha=0.7),
                    whiskerprops=dict(linewidth=2, alpha=0.7),
                    capprops=dict(linewidth=2, alpha=0.7),
                    medianprops=dict(linewidth=0, color='r'), 
                    flierprops=dict(markersize=8, linewidth=2), 
                    showmeans=True, meanprops=dict(markersize=10, color='m'));
for i, p in enumerate([0.1, 0.3, 0.5, 0.7, 0.9, 1.]):
    bp = axes[1, 0].boxplot(np.asarray(geno_first_dose[(p, 1.)]), positions=[i], widths=[0.2], 
                    boxprops=dict(linewidth=2, alpha=0.7),
                    whiskerprops=dict(linewidth=2, alpha=0.7),
                    capprops=dict(linewidth=2, alpha=0.7),
                    medianprops=dict(linewidth=0, color='r'), 
                    flierprops=dict(markersize=8, linewidth=2), 
                    showmeans=True, meanprops=dict(markersize=10, color='m'));
    bp = axes[1, 1].boxplot(np.asarray(geno_first_dose[(1., p)]), positions=[i], widths=[0.2], 
                    boxprops=dict(linewidth=2, alpha=0.7),
                    whiskerprops=dict(linewidth=2, alpha=0.7),
                    capprops=dict(linewidth=2, alpha=0.7),
                    medianprops=dict(linewidth=0, color='r'), 
                    flierprops=dict(markersize=8, linewidth=2), 
                    showmeans=True, meanprops=dict(markersize=10, color='m'));

axes[0, 0].set_title("p_c varies, p_m=0", fontsize=20)
axes[0, 1].set_title("p_m varies, p_c=0", fontsize=20)
axes[1, 0].set_title("p_c varies, p_m=1", fontsize=20)
axes[1, 1].set_title("p_m varies, p_c=1", fontsize=20)

for i in range(2):
    for j in range(2):
        axes[i, j].grid()
        axes[i, j].set_xticks([0, 1, 2, 3, 4, 5], [0.1, 0.3, 0.5, 0.7, 0.9, 1.])
        axes[i, j].set_xlabel("Probability", fontsize=20)
        axes[i, j].tick_params(axis='both', labelsize=15)
        # axes[i].legend([bp['medians'][0], bp['means'][0]], ['median', 'mean'], fontsize=20);
        axes[i, j].legend([bp['means'][0]], ['mean'], fontsize=20);
axes[0, 0].set_ylabel("Generation", fontsize=20);
axes[1, 0].set_ylabel("Generation", fontsize=20);